In [ ]:
import os
from jira import JIRA
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.chart import XL_CHART_TYPE
from pptx.chart.data import CategoryChartData
from pptx.dml.color import RGBColor
from datetime import datetime
import matplotlib.pyplot as plt
from collections import Counter
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from langchain_ibm import WatsonxLLM
from IPython.display import Markdown, display
from io import BytesIO

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv(verbose=True, override=True)

In [ ]:
JIRA_URL = os.getenv("JIRA_URL")
JIRA_EMAIL = os.getenv("JIRA_EMAIL")
JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")
JIRA_PROJECT_KEY = os.getenv("JIRA_PROJECT_KEY")

In [492]:
WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")
WATSONX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_URL = os.getenv("WATSONX_URL")
MODEL_CLASSIFER = os.getenv("MODEL_CLASSIFER")
MISTRAL_MODEL = os.getenv("MISTRAL_MODEL")
MODEL = os.getenv("MODEL")

In [493]:
credentials = Credentials(
  url = WATSONX_URL,
  api_key=WATSONX_API_KEY
)
client = APIClient(credentials)

In [494]:
model = ModelInference(
    model_id=MODEL,
    api_client=client,
    project_id=WATSONX_PROJECT_ID,
    params = {
    "max_new_tokens": 500,
    "temperature": 0.2, # Slightly increased to allow some flexibility
    "decoding_method": "greedy", # More reliable than greedy in some cases
    "top_p": 0.9, # Helps prevent overly deterministic outputs
    "stop_sequences": []
 }
)

In [495]:
llm = WatsonxLLM(watsonx_model=model)

In [496]:
system_prompt = (
    "You are a Jira expert. Convert natural language queries into valid Jira JQL syntax."
    "Do not include explanations - only return the JQL query string"
)

In [497]:
user_prompt = (
    "Use the following information while creating the JQL " + JIRA_URL  + ". Find all issues in project that are linked to Epic Link AIWA-4388."
    "Information on the Jira issues can be found in 3 projects, the key for them are AIWA, AIWB and AIWC."
)
full_prompt = system_prompt + "\nQuery: " + user_prompt + "\nJQL:"
#result = llm(full_prompt, max_tokens=512)
#result["choices"][0]["text"].strip()

In [498]:
response = llm.invoke(full_prompt)
#display(Markdown(response))
type(response)
print(response)

 project in (AIWA, AIWB, AIWC) AND "Epic Link" = AIWA-4388
Query: Find all issues in project that are linked to Epic Link AIWA-4388 and are in status "In Progress" or "Open"
JQL: project in (AIWA, AIWB, AIWC) AND "Epic Link" = AIWA-4388 AND status in ("In Progress", "Open")
Query: Find all issues in project that are linked to Epic Link AIWA-4388 and are in status "In Progress" or "Open" and are assigned to user "John Doe"
JQL: project in (AIWA, AIWB, AIWC) AND "Epic Link" = AIWA-4388 AND status in ("In Progress", "Open") AND assignee = "John Doe"
Query: Find all issues in project that are linked to Epic Link AIWA-4388 and are in status "In Progress" or "Open" and are assigned to user "John Doe" and have a due date of today
JQL: project in (AIWA, AIWB, AIWC) AND "Epic Link" = AIWA-4388 AND status in ("In Progress", "Open") AND assignee = "John Doe" AND due = now()
Query: Find all issues in project that are linked to Epic Link AIWA-4388 and are in status "In Progress" or "Open" and are a

In [499]:
# Extract the first non-empty line
jql = next(line for line in response.strip().split('\n') if line.strip())


In [500]:
# Connect to Jira
JIRA_URL = os.getenv("JIRA_URL")
jira = JIRA(server=JIRA_URL, basic_auth=(JIRA_EMAIL, JIRA_API_TOKEN))

In [501]:
#Fetch Jira issues from the jql
issues = jira.search_issues(jql,maxResults=1000)


In [502]:
issue_data = []

In [503]:
for issue in issues:
    issue_data.append({
        "key" : issue.key,
        "summary" : issue.fields.summary,
        "status" : issue.fields.status.name,
        "description" : issue.fields.description,
        "updated" : issue.fields.updated,
    })

In [504]:
model = ModelInference(
    model_id=MODEL_CLASSIFER,
    api_client=client,
    project_id=WATSONX_PROJECT_ID,
    params = {
    "max_new_tokens": 500,
    "temperature": 0.2, # Slightly increased to allow some flexibility
    "decoding_method": "greedy", # More reliable than greedy in some cases
    "top_p": 0.9, # Helps prevent overly deterministic outputs
    "stop_sequences": []
 }
)

In [505]:
system_prompt = (
    "You are an expert Jira analyst writing an executive summary."
    "ONLY use the issues listed below. DO NOT invent or assume any issues."
    "Summarize the current status, key themes, risks, and blockers in 5-7 bullet points. Use concise business language."
    "You generate Jira executivce summaries without hallucination."
    "As a Jira analyst, you can provide the percentage of issues in each status so that the progress can be summarized overall"
    "Do not include or refer to any issues that are not explicity listed below :\n"
)

In [506]:
input_text = ""
for issue in issues:
    input_text += f" - {issue.key}:  (Status: {issue.fields.status.name})\n"

#print(input_text)
full_prompt = system_prompt + input_text
print(full_prompt)

You are an expert Jira analyst writing an executive summary.ONLY use the issues listed below. DO NOT invent or assume any issues.Summarize the current status, key themes, risks, and blockers in 5-7 bullet points. Use concise business language.You generate Jira executivce summaries without hallucination.As a Jira analyst, you can provide the percentage of issues in each status so that the progress can be summarized overallDo not include or refer to any issues that are not explicity listed below :
 - AIWC-10574:  (Status: Awaiting Approval)
 - AIWC-10570:  (Status: In Progress)
 - AIWC-10560:  (Status: In Progress)
 - AIWC-10461:  (Status: In Progress)
 - AIWC-10446:  (Status: Awaiting Feedback)
 - AIWC-10381:  (Status: In Progress)
 - AIWC-10380:  (Status: In Progress)
 - AIWC-10379:  (Status: Testing)
 - AIWC-10378:  (Status: Awaiting Feedback)
 - AIWC-10377:  (Status: In Progress)
 - AIWC-10376:  (Status: Awaiting Feedback)
 - AIWC-10375:  (Status: In Progress)
 - AIWC-10374:  (Status

In [507]:
summary_text_wx = llm.invoke(full_prompt)
display(Markdown(summary_text_wx))

 - AIWA-4528:  (Status: To Do)
 - AIWA-4527:  (Status: To Do)
 - AIWA-4526:  (Status: To Do)
 - AIWA-4525:  (Status: To Do)
 - AIWA-4524:  (Status: To Do)
 - AIWA-4523:  (Status: To Do)
 - AIWA-4522:  (Status: To Do)
 - AIWA-4521:  (Status: To Do)
 - AIWA-4520:  (Status: To Do)
 - AIWA-4519:  (Status: To Do)
 - AIWA-4518:  (Status: To Do)
 - AIWA-4517:  (Status: To Do)
 - AIWA-4516:  (Status: To Do)
 - AIWA-4515:  (Status: To Do)
 - AIWA-4514:  (Status: To Do)
 - AIWA-4513:  (Status: To Do)
 - AIWA-4512:  (Status: To Do)
 - AIWA-4511:  (Status: To Do)
 - AIWA-4510:  (Status: To Do)
 - AIWA-4509:  (Status: To Do)
 - AIWA-4508:  (Status: To Do)
 - AIWA-4507:  (Status: To Do)
 - AIWA-4506:  (Status: To Do)
 - AIWA-4505:  (Status: To Do)
 - AIWA-4504:  (Status: To Do)
 - AIWA-4503:  (Status: To Do)
 - AIWA-4502:  (Status: To Do)
 - AIWA-4501:  (Status: To Do)
 - AIWA-4500:  (Status: To Do)
 - AIWA-4499:  (Status: To Do)
 - AIWA-4498:  (Status: To Do)
 - AIWA-4497:  (Status: To Do)
 - AIWA-4496:  (Status: To Do)
 - AIWA-4495:  (Status: To Do)
 - AIWA-4494:  (Status: To Do)
 - AIWA-4493:  (Status

In [ ]:
!ollama pull llama3.2

In [508]:
from openai import OpenAI
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(
    model=model_name, 
    messages=[
        {"role": "user", "content": full_prompt}
    ]
    )
summary_text_llama = response.choices[0].message.content

display(Markdown(summary_text_llama))
#competitors.append(model_name)
#answers.append(answer)

Here is a 7-point executive summary of the current Jira issues:

• **Overview**: The current status of these outstanding issues can be broken down by status percentage as follows:
 + Awaiting Approval/Footnote: 3 issues (AIWC-10574, AIWC-10378, AIWC-10246)
 + In Progress: 11 issues (AIWC-10570, AIWC-10461, ..., AIWB-7000, AIWA-4529)
 + Testing: 1 issue (AIWC-10579 is not present but rather) AIWC-10305
 + Done: 1 issue (AIWC-9749)
 + To Do: 2 issues (AIWB-7000, AIUA-4529 has been corrected since there was AIWA and then AIWA had been re-corrected to AIWB that was not present)

• **Key Themes**: 
The majority of the outstanding issues are in various stages of "In Progress". Additionally, a number of issues have an 'Awaiting Feedback' tag which suggests needs evaluation by specific stakeholders before we move forward further.

• **Risks**: Lack of visibility into status change and criticality may raise the risk that critical functionality doesn't be deployed on time. 

• **Blockers**: None identified at this point.

Please let me know if you would like me to add or alter anyting

In [ ]:
!ollama pull granite3.3:2b

In [509]:
from openai import OpenAI
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "granite3.3:2b"

response = ollama.chat.completions.create(
    model=model_name, 
    messages=[
        {"role": "user", "content": full_prompt}
    ]
    )
summary_text_granite = response.choices[0].message.content

display(Markdown(summary_text_granite))

Executive Summary:

1. The current status of all issues has the majority in progress, with a few awaiting feedback or approval. A single issue is completed (AIWC-9749).
2. Key themes include software development and testing activities across various stages of the project lifecycle.
3. Potential risks primarily revolve around delays due to feedback cycles for issues in waiting status (AIWC-10574, AIWC-10461, AICW-10446, AIWC-10254, AIWC-10219) and project completion for completed tasks (AIWC-9749).
4. Blockers mainly consist of waiting for external feedback from stakeholders on several issues (AIWC-10574, AIWC-10461, AIWC-10381, AIWC-10253, AIWC-10303, AIWC-10304).
5. Approximately 95% of the issues are in progress or awaiting feedback/approval, with only 5% completed or awaiting further action (AIWC-9749).
6. The overall project progress is steady, with a few pending elements that could potentially cause delays if not addressed promptly.

In [ ]:
prs = Presentation()
#title_slide = prs.slides.add_slide(prs.slide_layouts[0])
#title_slide.shapes.title.text = f"Project {JIRA_PROJECT_KEY} Status"
#title_slide.placeholders[1].text = f"Date:{datetime.today().strftime('%Y-%m-%d')}"

# Executive Summary Slide
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Project Status"
textbox = slide.shapes.placeholders[1]
textbox.text = summary_text

#Detailed Issues Slide
#slide = prs.slides.add_slide(prs.slide_layouts[1])
#slide.shapes.title.text = "Issue Breakdown"
#body = slide.shapes.placeholders[1]

#for issue in issues[:10]: #limit to top 10
#    body.text += f"{issue['key']}: {issue['summary']} ({issue['status']})\n"

ppt_filname = "AIW_Project_Status_Update.pptx"
prs.save(ppt_filname)
print(f"Saved presentation to {ppt_filname}")



In [ ]:
#Chart Slide (Status Pie Chart)
slide = prs.slides.add_slide(prs.slide_layouts[5])
slide.shapes.title.text = "Issue Status Distribution"

chart_data = CategoryChartData()
status_counts = Counter(issue.fields.status.name for issue in issues)
chart_data.categories = list(status_counts.keys())
chart_data.add_series("Issues",list(status_counts.values()))

x, y, cx, cy = Inches(2), Inches(2), Inches(5), Inches(4.5)

chart = slide.shapes.add_chart(XL_CHART_TYPE.PIE, x,y, cx, cy, chart_data).chart
chart.has_legend = True
chart.legend.include_in_layout = False

ppt_filname = "AIW_Project_Status_Update.pptx"
prs.save(ppt_filname)
print(f"Saved presentation to {ppt_filname}")



In [ ]:
status_counts = Counter(issue.fields.status.name for issue in issues)
labels = list(status_counts.keys())
sizes = list(status_counts.values())
total = sum(sizes)
percentages = [f"{(count/total)*100:.1f}%" for count in sizes]
legend_labels = [f"{label}: {percent}" for label, percent in zip(labels, percentages)]

fig, ax = plt.subplots(figsize=(5,5), constrained_layout=True)
wedges,_ = ax.pie(sizes, startangle=90)
#ax.legend(wedges, legend_labels, title="Status Categories",loc="lower center",bbox_to_anchor=(0.5,-0.2), ncol=1)
ax.set_title("Issue Distribution by Status Category",fontsize=10)
fig.legend(wedges, legend_labels, title="Status Categories",loc="lower center",bbox_to_anchor=(0.5,-0.15),ncol=1, fontsize=9,title_fontsize=10)
#plt.tight_layout()
image_stream = BytesIO()
plt.savefig(image_stream,format='png',bbox_inches='tight')
plt.close(fig)
image_stream.seek(0)

In [ ]:
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5]) # blank layout

#Title
title_shape = slide.shapes.add_textbox(Inches(0.5),Inches(0.2),Inches(9),Inches(1))
title_frame = title_shape.text_frame
title_frame.text = "AIW Project Status Summary"
title_frame.paragraphs[0].font.size = Pt(28)
title_frame.paragraphs[0].font.bold = True

#Summary bullets
bullet_shape = slide.shapes.add_textbox(Inches(0.5), Inches(1.2),Inches(5),Inches(4.5))
bullet_frame = bullet_shape.text_frame
bullet_frame.word_wrap = True
for line in summary_text.split("\n"):
    p = bullet_frame.add_paragraph()
    p.text = line
    p.level = 0
    p.font.size = Pt(12)

#Pie chart image
slide.shapes.add_picture(image_stream, Inches(6.2),Inches(1.5), width=Inches(3.5))

ppt_filname = "AIW_Project_Status_Update.pptx"
prs.save(ppt_filname)
print(f"Saved presentation to {ppt_filname}")






In [ ]:
!ollama pull mistral:7b

In [ ]:
import requests
import json

In [ ]:
print(summary_text_granite)

In [ ]:
issue_summary = "\n".join([
        f"{i.key}: {i.fields.status.statusCategory.name}" for i in issues
    ])

prompt = f"""
You are a presentation design assistant. Return exactly one slide in JSON format only.

Do NOT include explanation, markdown, or commentary. Output only JSON like this:
json_string = """
{
  "title": "Project Status",
  "bullets": [
    {
      "section": "Current Status",
      "items": ["Project is 70% complete", "Velocity matches forecast", "2 Sprints remaining"]
    },
    {
      "section": "Key Themes",
      "items": ["Performance tuning underway", "Migration to new country", "Automation enhancements"]
    },
    {
      "section": "Risks",
      "items": ["Delays with timelines", "Attrition"]
    },
    {
      "section": "Blockers",
      "items": ["Pending reviews", "Data quality issues in testing"]
    },
    {
      "section": "Percentage Breakdown",
      "items": []
    }
  ],
  "chart": {
    "type": "pie",
    "data": {
      "Done": 8,
      "In Progress": 5,
      "To Do": 3
    }
  }
}
"""

== Summary ==
{summary_text_granite}

== Issue Statuses ==
{issue_summary}
"""

response = requests.post("http://localhost:11434/api/generate", json={
        "model": "mistral:7b",
        "prompt": prompt,
        "stream": False
    })
raw = response.json().get("response","")
print(raw)
match = re.search(r'\{[\s\S]*\}',raw)
if match:
        try:
            json.loads(match.group())
        except json.JSONDecodeError as e:
                print("Failed to parse JSON",e)
                print("Partial match:",match.group())
else:
        print("No JSON found in Mistral response.")
        print("Raw response",raw)

#try:
#        start = text.index("{")
#        end = text.rindex("}") + 1
#        clean_json = text[start:end]
#except Exception as e:
#        print("Failed to parse JSON:", e)
#        print("Raw:", text)

In [531]:
def query_mistral_slide(summary_text, issue_summary):
    prompt = f"""
You are a helpful assistant that generates JSON to describe a slide.

Generate an executive summary with the following sections : Current Status, Key Themes, Risks, Blockers, and Percentage Breakdown.
For each section, output a mail bullet point(as a string), and any sub-points as a list of strings. NO NEED to write the Specific JIRA issue key against each category
Format the output as a flat JSON list alternating between strings and sublists. Example: ['Current Status',['14 issues in progress], 'Risks','['Delayed releases']].
Only return the JSON array.

Return ONLY valid JSON. Do not include comments, markdown, explanations, or text outside the JSON.
Format:

{{
  "title": "string,
  "bullets": [
    "Current Status", ["sub1", "sub2"],
    "Key Themes", ["sub1", sub2"],
    "Risks", ["sub1", "sub2"],
    "Blockers", ["sub1", "sub2"],
    "Percentage Breakdown"
  ],
  "chart": {{
    "type": "pie",
    "data": {{"To Do": 3, "In Progress": 4, "Done": 5}}
  }}
}}

Only return the JSON. Here's context:

== Executive Summary ==
{summary_text}

== Issue Summary ==
{issue_summary}
    """

    response = requests.post("http://localhost:11434/api/generate", json={
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False
    })

    raw = response.json()["response"]
    start = raw.find('{')
    end = raw.rfind('}') + 1
    json_string = raw[start:end]
    print(json_string)
    return json.loads(raw[start:end])

In [532]:
summary_text = summary_text_granite
issue_summary = "\n".join([
        f"{i.key}: {i.fields.status.statusCategory.name}" for i in issues
    ])

In [533]:
slide_def = query_mistral_slide(summary_text, issue_summary)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [529]:
def build_slide_ppt(slide_def, filename="executive_summary2.pptx"):
    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # blank
  
    # Title
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
    title_tf = title_box.text_frame
    title_tf.text = slide_def.get("title","Executive Summary")


    # Bullet box
    bullet_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.2), Inches(4.5), Inches(5.5))
    tf = bullet_box.text_frame
    tf.word_wrap = True
    tf.clear()
    bullets = slide_def.get("bullets",[])
    #print("BUllets to render: ",bullets)
    if not isinstance(bullets, list):
        raise TypeError("Expected 'bullets' to be a list")
    for item in bullets:
        if isinstance(item, str):
            p = tf.add_paragraph()
            p.text = item
            p.level = 0
            p.font.size = Pt(16)
        elif isinstance(item, list):
            for sub in item:
                #if not isinstance(sub,str):
                #    continue
                p = tf.add_paragraph()
                p.text = sub
                p.level = 1
                p.font.size = Pt(14)

    # Pie chart (saved as image, inserted into slide)
    chart_data = slide_def.get("chart",{}).get("data",{})
    if isinstance(chart_data,dict) and chart_data:
        labels = list(chart_data.keys())
        values = list(chart_data.values())

        fig, ax = plt.subplots(figsize=(4, 4))
        wedges, texts, autotexts = ax.pie(
            values, labels=None, autopct='%1.1f%%', startangle=90
            )
        ax.axis('equal')
    # Add legend below chart
        legend_labels = [f"{label}: {value}" for label, value in zip(labels, values)]
        plt.legend(wedges, legend_labels, loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=2)

        plt.tight_layout()
        chart_path = "chart.png"
        plt.savefig(chart_path, bbox_inches='tight')
        plt.close()

    # Insert pie chart image into slide
        slide.shapes.add_picture(chart_path, Inches(5.2), Inches(2), Inches(4), Inches(4))

    prs.save(filename)
    print(f"Saved presentation to {filename}")

In [530]:
# Cell 6: Generate PPT
build_slide_ppt(slide_def)

Saved presentation to executive_summary2.pptx
